# Category-based Error Analysis

This notebook analyzes weight/volume estimation errors grouped by category type:
- **Meaningful**: Valid product categories with proper hierarchy (big > sub > detailed)
- **Malformed**: Invalid categories (navigation, promotional, undefined, etc.)
- **Empty**: Items with no category

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from pathlib import Path

# Set matplotlib style with fallback
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except:
    try:
        plt.style.use('seaborn-whitegrid')
    except:
        pass  # Use default style

plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("Setup complete!")

## Load Data

**For Google Colab**: Run this cell to upload the TSV file.

In [ ]:
# Check if running in Colab
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Google Colab. Please upload '20260128_experiment_datasource.tsv'")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    print(f"Uploaded: {filename}")
else:
    filename = '20260128_experiment_datasource.tsv'
    print(f"Running locally. Using: {filename}")

In [ ]:
# Load the experiment datasource
df = pd.read_csv(filename, sep='\t')
print(f"Data loaded successfully!")
print(f"\nTotal rows: {len(df):,}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

## Data Cleaning & Validation

Check data completeness and calculate missing error values.

In [ ]:
# Check data completeness
print("DATA COMPLETENESS CHECK")
print("=" * 60)

# Actual values
actual_cols = ['actual_weight', 'actual_d1', 'actual_d2', 'actual_d3', 'actual_volume_cm3']
print("\nActual Values (Ground Truth):")
for col in actual_cols:
    if col in df.columns:
        valid = df[col].notna().sum()
        print(f"  {col:20s}: {valid:6,} valid ({valid/len(df)*100:.1f}%)")

# AI estimated values
ai_cols = ['ai_weight_kg', 'ai_width_cm', 'ai_depth_cm', 'ai_height_cm', 'ai_volume_cm3']
print("\nAI Estimated Values:")
for col in ai_cols:
    if col in df.columns:
        valid = df[col].notna().sum()
        print(f"  {col:20s}: {valid:6,} valid ({valid/len(df)*100:.1f}%)")

# Error columns
error_cols = ['weight_error', 'volume_error']
print("\nError Columns:")
for col in error_cols:
    if col in df.columns:
        valid = df[col].notna().sum()
        print(f"  {col:20s}: {valid:6,} valid ({valid/len(df)*100:.1f}%)")

In [ ]:
# Calculate missing values
print("CALCULATING MISSING VALUES")
print("=" * 60)

# 1. Calculate AI volume if missing
if 'ai_volume_cm3' not in df.columns or df['ai_volume_cm3'].isna().all():
    print("\n1. Calculating ai_volume_cm3 from dimensions...")
    df['ai_volume_cm3'] = df['ai_width_cm'] * df['ai_depth_cm'] * df['ai_height_cm']
    valid = df['ai_volume_cm3'].notna().sum()
    print(f"   Created ai_volume_cm3: {valid:,} valid values")
else:
    # Fill missing values
    missing = df['ai_volume_cm3'].isna().sum()
    if missing > 0:
        print(f"\n1. Filling {missing:,} missing ai_volume_cm3 values...")
        df['ai_volume_cm3'] = df['ai_volume_cm3'].fillna(
            df['ai_width_cm'] * df['ai_depth_cm'] * df['ai_height_cm']
        )

# 2. Calculate volume_error if missing
# Formula: (estimated - actual) / actual
if 'volume_error' not in df.columns or df['volume_error'].isna().all():
    print("\n2. Calculating volume_error...")
    df['volume_error'] = (df['ai_volume_cm3'] - df['actual_volume_cm3']) / df['actual_volume_cm3']
    valid = df['volume_error'].notna().sum()
    print(f"   Created volume_error: {valid:,} valid values")
else:
    missing = df['volume_error'].isna().sum()
    if missing > 0:
        print(f"\n2. Filling {missing:,} missing volume_error values...")
        df['volume_error'] = df['volume_error'].fillna(
            (df['ai_volume_cm3'] - df['actual_volume_cm3']) / df['actual_volume_cm3']
        )

# 3. Verify weight_error exists
if 'weight_error' not in df.columns or df['weight_error'].isna().all():
    print("\n3. Calculating weight_error...")
    df['weight_error'] = (df['ai_weight_kg'] - df['actual_weight']) / df['actual_weight']
    valid = df['weight_error'].notna().sum()
    print(f"   Created weight_error: {valid:,} valid values")

print("\n" + "=" * 60)
print("DATA AFTER CLEANING:")
print(f"  weight_error valid: {df['weight_error'].notna().sum():,}")
print(f"  volume_error valid: {df['volume_error'].notna().sum():,}")
print(f"  Both valid:         {(df['weight_error'].notna() & df['volume_error'].notna()).sum():,}")

## Category Classification Functions

In [ ]:
def is_empty_category(cat):
    """Check if category is empty"""
    if pd.isna(cat):
        return True
    return not str(cat).strip()

def is_malformed_category(cat):
    """Check if category is malformed (not a proper product hierarchy)"""
    if is_empty_category(cat):
        return False
    
    cat = str(cat)
    
    # 1. Contains 'undefined'
    if 'undefined' in cat.lower():
        return True
    
    # 2. Template/code patterns
    template_patterns = ['_imgTag_', '_html_', '_name_', '{#item}', '{#title}', '{$', '_count}']
    for p in template_patterns:
        if p in cat:
            return True
    
    # 3. Just a dash
    if cat in ['-', '--', '---']:
        return True
    
    # 4. Contains parentheses with numbers in navigation context
    if re.search(r'\(\s*\d+\s*\)', cat):
        nav_indicators = ['로그인', '회원가입', '장바구니', '카트', 'cart', 'Cart', 'CART', 
                          '마이페이지', 'Q&A', 'FAQ', 'REVIEW', '상품후기', '상품문의', 
                          '상세정보', '기본정보', 'DETAIL', 'GUIDE']
        for indicator in nav_indicators:
            if indicator in cat:
                return True
    
    # 5. Navigation/UI/Account keywords
    nav_only_patterns = [
        '로그인', '회원가입', '마이페이지', '마이쇼핑', '주문조회', 
        '카트 ', 'cart ', 'Cart ', 'LOGIN', 'JOIN', 'CART', 'MYPAGE', 'MY ACCOUNT',
        'MY PAGE', 'ORDER CHECK', 'YOUR BAG', 'Sign In', 'Sign Up',
        '바로가기', '닫기 >', '뒤로가기', '뒤로 >', '이전 페이지',
        '인기 검색어', '최근 검색어', 'Press Enter',
        'Q&A', 'FAQ', 'NOTICE', '리뷰이벤트', '공지사항', '판매자 정보',
        '상세설명 >', '상세정보 >', '기본정보', '구매안내', 'DETAIL >', 'GUIDE >',
        'INFO >', 'SIZE CHART', 'WASHING', 'DELIVERY', 'EXCHANGE', 'RETURN',
        '비회원', '아이디 찾기', '비밀번호 찾기',
        'SHARE', 'URL >', 'kakaostory', 'kakao >', 'instagram', 'INSTAGRAM'
    ]
    for kw in nav_only_patterns:
        if kw in cat:
            return True
    
    # 6. Promotional/marketing patterns
    promo_patterns = [
        '쿠폰', '할인쿠폰', ' SALE', ' sale', '블프', 'BLACK FRIDAY',
        'DROP', 'OPEN >', 'PRE-ORDER', '📢', '⏰', '⭐', '💐', '🍓', '⛄', '♥', '✦',
        '무료배송', '웰컴', '멤버십', 'MEMBERSHIP', 'Membership',
        '신규회원', '가입 시', '추가 시', '% off', 
        '~90%', '~70%', '~60%', '~50%', '~40%', '~30%', '~20%', '~10%',
        'COLLECTION. SEE MORE', 'SEE MORE', 'SHOP NOW',
        '100원', '3,000원', 'PICK♥', 'PICK >'
    ]
    for p in promo_patterns:
        if p in cat:
            return True
    
    # 7. Shipping/payment context
    if any(x in cat for x in ['배송 방법', '배송방법', '배송 지역', '배송 비용', 
                               '배송 안내', '결제 시', '무통장', '입금 확인',
                               '반품/교환', '교환/반품', 'a/s안내', 'A/S안내', '상품상세참조',
                               '연관상품', '수수료', '거주', '셀러']):
        return True
    
    # 8. Single/incomplete categories ending with >
    parts = [p.strip() for p in cat.split('>')]
    non_empty_parts = [p for p in parts if p]
    
    if cat.strip().endswith('>'):
        if len(non_empty_parts) <= 1:
            return True
        if len(non_empty_parts) == 2 and non_empty_parts[0] in [
            '홈', 'HOME', 'Home', 'home', '기타', 'Detail', 'TOP', 'HEADWEAR',
            'PRODUCTS', 'MOMENTEL', 'NICHES', 'YOOKIESHOT', 'BRAND', 'Menu',
            'close', 'more', 'Today', 'o w o n', 'PETER AND WENDY'
        ]:
            return True
    
    # 9. Specific garbage patterns
    garbage_patterns = [
        'ourcabinet', '소득공제 >', 'AW24 New Arrival SHOP NOW',
        '24FW NEW COLLECTION >', '365일초특가', '슈피겐 공지사항',
        '듀이 멤버십', '지공가능한회원샵', '노벨문학상 >', 'SALE샵',
        '# 검색어', 'COMPANY > AGREEMENT', 'shop > story > sale',
        '1 . 컬리지', '전체상품 > 산리오', 'About > Archive > Wholesale',
        '쿠팡 홈 > 쿠팡수입', '★주말특가'
    ]
    for p in garbage_patterns:
        if p in cat:
            return True
    
    # 10. Very short standalone tokens
    if cat in ['HOME', 'SHOP', 'ALL', 'BEST', 'NEW', 'TOP', 'OUTER', 'BOTTOM', 
               'HEADWEAR', 'PRODUCTS', 'BRAND', 'GOODS', 'ACC']:
        return True
    
    return False

def classify_category(cat):
    """Classify category into: empty, malformed, or meaningful"""
    if is_empty_category(cat):
        return 'empty'
    elif is_malformed_category(cat):
        return 'malformed'
    else:
        return 'meaningful'

print("Classification functions defined!")

## Classify Categories

In [ ]:
# Classify each row
df['category_type'] = df['category'].apply(classify_category)

# Summary
category_counts = df['category_type'].value_counts()
print("Category Type Distribution:")
print("=" * 40)
for cat_type in ['meaningful', 'malformed', 'empty']:
    count = category_counts.get(cat_type, 0)
    pct = count / len(df) * 100
    print(f"{cat_type:12s}: {count:6,} ({pct:5.1f}%)")
print("=" * 40)
print(f"{'Total':12s}: {len(df):6,}")

## Prepare Error Data

In [ ]:
# Set standard column names
WEIGHT_ERROR = 'weight_error'
VOLUME_ERROR = 'volume_error'

# Filter rows with valid errors (both weight and volume)
df_valid = df.dropna(subset=[WEIGHT_ERROR, VOLUME_ERROR]).copy()
print(f"Rows with valid weight_error AND volume_error: {len(df_valid):,}")

# Create combined error (average of absolute errors)
df_valid['combined_error'] = (df_valid[WEIGHT_ERROR].abs() + df_valid[VOLUME_ERROR].abs()) / 2

# Show distribution by category type
print("\nValid rows by category type:")
print(df_valid['category_type'].value_counts())

## Error Statistics by Category Type

In [ ]:
def calc_error_stats(data):
    """Calculate error statistics for a series"""
    data = data.dropna()
    if len(data) == 0:
        return pd.Series({
            'count': 0, 'min': np.nan, 'max': np.nan, 
            'median': np.nan, 'mean': np.nan, 'std': np.nan,
            'q25': np.nan, 'q75': np.nan
        })
    return pd.Series({
        'count': len(data),
        'min': data.min(),
        'max': data.max(),
        'median': data.median(),
        'mean': data.mean(),
        'std': data.std(),
        'q25': data.quantile(0.25),
        'q75': data.quantile(0.75)
    })

# Calculate stats for each category type and error type
error_types = [WEIGHT_ERROR, VOLUME_ERROR, 'combined_error']
cat_order = ['meaningful', 'malformed', 'empty']

for error_type in error_types:
    print(f"\n{'='*60}")
    print(f"{error_type.upper()} Statistics by Category Type")
    print(f"{'='*60}")
    
    stats_list = []
    for cat_type in cat_order:
        subset = df_valid[df_valid['category_type'] == cat_type][error_type]
        stats = calc_error_stats(subset)
        stats.name = cat_type
        stats_list.append(stats)
    
    stats_df = pd.DataFrame(stats_list)
    print(stats_df.round(4).to_string())

## Visualize Error Distribution by Category Type

### Box Plots

In [ ]:
# Color palette for category types
colors = {'meaningful': '#2ecc71', 'malformed': '#e74c3c', 'empty': '#95a5a6'}

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

titles = ['Weight Error', 'Volume Error', 'Combined Error']

for ax, error_type, title in zip(axes, error_types, titles):
    # Prepare data for box plot
    data_to_plot = []
    labels = []
    for cat_type in cat_order:
        subset = df_valid[df_valid['category_type'] == cat_type][error_type].dropna()
        # Clip extreme values for better visualization
        subset_clipped = subset.clip(-5, 5)
        data_to_plot.append(subset_clipped.values)
        labels.append(f"{cat_type}\n(n={len(subset):,})")
    
    # Box plot
    bp = ax.boxplot(data_to_plot, labels=labels, patch_artist=True,
                    showfliers=False, widths=0.6)
    
    # Color the boxes
    for patch, cat_type in zip(bp['boxes'], cat_order):
        patch.set_facecolor(colors[cat_type])
        patch.set_alpha(0.7)
    
    # Add horizontal line at 0
    ax.axhline(y=0, color='black', linestyle='--', alpha=0.5, linewidth=1)
    
    # Add median annotations
    for i, cat_type in enumerate(cat_order):
        subset = df_valid[df_valid['category_type'] == cat_type][error_type]
        if len(subset) > 0:
            median = subset.median()
            ax.annotate(f'med: {median:.3f}', xy=(i+1, median), 
                        xytext=(i+1.3, median), fontsize=9,
                        ha='left', va='center')
    
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_ylabel('Error (clipped to [-5, 5])')
    ax.set_ylim(-5.5, 5.5)
    ax.grid(axis='y', alpha=0.3)

plt.suptitle('Error Distribution by Category Type', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('category_error_boxplot.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nBox plot saved as 'category_error_boxplot.png'")

### Histogram and CDF Plots

In [ ]:
# Histogram comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for col, (error_type, title) in enumerate(zip(error_types, titles)):
    # Top row: full distribution (clipped)
    ax = axes[0, col]
    for cat_type in cat_order:
        subset = df_valid[df_valid['category_type'] == cat_type][error_type].dropna()
        if len(subset) > 0:
            subset_clipped = subset.clip(-3, 3)
            ax.hist(subset_clipped.values, bins=50, alpha=0.5, label=cat_type, 
                    color=colors[cat_type], density=True)
    ax.axvline(x=0, color='black', linestyle='--', alpha=0.7)
    ax.set_title(f'{title} Distribution', fontsize=12, fontweight='bold')
    ax.set_xlabel('Error (clipped to [-3, 3])')
    ax.set_ylabel('Density')
    ax.legend()
    ax.set_xlim(-3.5, 3.5)
    
    # Bottom row: cumulative distribution
    ax = axes[1, col]
    for cat_type in cat_order:
        subset = df_valid[df_valid['category_type'] == cat_type][error_type].abs().dropna()
        if len(subset) > 0:
            sorted_data = np.sort(subset.values)
            cumulative = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
            ax.plot(sorted_data, cumulative, label=cat_type, color=colors[cat_type], linewidth=2)
    ax.axvline(x=0.5, color='gray', linestyle=':', alpha=0.7, label='50% error')
    ax.axvline(x=1.0, color='gray', linestyle='--', alpha=0.7, label='100% error')
    ax.set_title(f'{title} CDF (Absolute)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Absolute Error')
    ax.set_ylabel('Cumulative Probability')
    ax.legend(loc='lower right')
    ax.set_xlim(0, 3)
    ax.set_ylim(0, 1)
    ax.grid(alpha=0.3)

plt.suptitle('Error Distribution Comparison by Category Type', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('category_error_histogram.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nHistogram saved as 'category_error_histogram.png'")

## Summary Statistics Table

In [ ]:
# Create a summary table
summary_data = []

for cat_type in cat_order:
    subset = df_valid[df_valid['category_type'] == cat_type]
    
    if len(subset) > 0:
        row = {
            'Category Type': cat_type,
            'Count': len(subset),
            'Weight Min': subset[WEIGHT_ERROR].min(),
            'Weight Max': subset[WEIGHT_ERROR].max(),
            'Weight Median': subset[WEIGHT_ERROR].median(),
            'Volume Min': subset[VOLUME_ERROR].min(),
            'Volume Max': subset[VOLUME_ERROR].max(),
            'Volume Median': subset[VOLUME_ERROR].median(),
            'Combined Median': subset['combined_error'].median()
        }
    else:
        row = {
            'Category Type': cat_type,
            'Count': 0,
            'Weight Min': np.nan, 'Weight Max': np.nan, 'Weight Median': np.nan,
            'Volume Min': np.nan, 'Volume Max': np.nan, 'Volume Median': np.nan,
            'Combined Median': np.nan
        }
    summary_data.append(row)

summary_df = pd.DataFrame(summary_data)
print("\nError Summary by Category Type:")
print("=" * 100)
print(summary_df.round(4).to_string(index=False))

## Visual Summary: Min, Median, Max by Category Type

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

x = np.arange(len(cat_order))
width = 0.25

# Weight Error
ax = axes[0]
mins = [df_valid[df_valid['category_type']==ct][WEIGHT_ERROR].min() if len(df_valid[df_valid['category_type']==ct]) > 0 else 0 for ct in cat_order]
medians = [df_valid[df_valid['category_type']==ct][WEIGHT_ERROR].median() if len(df_valid[df_valid['category_type']==ct]) > 0 else 0 for ct in cat_order]
maxs = [df_valid[df_valid['category_type']==ct][WEIGHT_ERROR].max() if len(df_valid[df_valid['category_type']==ct]) > 0 else 0 for ct in cat_order]

bars1 = ax.bar(x - width, mins, width, label='Min', color='#3498db', alpha=0.8)
bars2 = ax.bar(x, medians, width, label='Median', color='#2ecc71', alpha=0.8)
bars3 = ax.bar(x + width, maxs, width, label='Max', color='#e74c3c', alpha=0.8)

# Add value labels on bars
for bar, val in zip(bars1, mins):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{val:.2f}', 
            ha='center', va='bottom' if val >= 0 else 'top', fontsize=8)
for bar, val in zip(bars2, medians):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{val:.2f}', 
            ha='center', va='bottom' if val >= 0 else 'top', fontsize=8)
for bar, val in zip(bars3, maxs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{val:.2f}', 
            ha='center', va='bottom', fontsize=8)

ax.set_ylabel('Weight Error')
ax.set_title('Weight Error: Min, Median, Max by Category Type', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(cat_order)
ax.legend()
ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax.grid(axis='y', alpha=0.3)

# Volume Error
ax = axes[1]
mins = [df_valid[df_valid['category_type']==ct][VOLUME_ERROR].min() if len(df_valid[df_valid['category_type']==ct]) > 0 else 0 for ct in cat_order]
medians = [df_valid[df_valid['category_type']==ct][VOLUME_ERROR].median() if len(df_valid[df_valid['category_type']==ct]) > 0 else 0 for ct in cat_order]
maxs = [df_valid[df_valid['category_type']==ct][VOLUME_ERROR].max() if len(df_valid[df_valid['category_type']==ct]) > 0 else 0 for ct in cat_order]

bars1 = ax.bar(x - width, mins, width, label='Min', color='#3498db', alpha=0.8)
bars2 = ax.bar(x, medians, width, label='Median', color='#2ecc71', alpha=0.8)
bars3 = ax.bar(x + width, maxs, width, label='Max', color='#e74c3c', alpha=0.8)

# Add value labels on bars
for bar, val in zip(bars1, mins):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{val:.2f}', 
            ha='center', va='bottom' if val >= 0 else 'top', fontsize=8)
for bar, val in zip(bars2, medians):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{val:.2f}', 
            ha='center', va='bottom' if val >= 0 else 'top', fontsize=8)
for bar, val in zip(bars3, maxs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{val:.2f}', 
            ha='center', va='bottom', fontsize=8)

ax.set_ylabel('Volume Error')
ax.set_title('Volume Error: Min, Median, Max by Category Type', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(cat_order)
ax.legend()
ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('category_error_summary.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nSummary chart saved as 'category_error_summary.png'")

## Separate Datasources by Category Type

Create separate TSV files for each category type for future experiments.

In [ ]:
# Separate and save
output_files = {}

for cat_type in cat_order:
    subset = df[df['category_type'] == cat_type].copy()
    filename = f'datasource_{cat_type}.tsv'
    subset.to_csv(filename, sep='\t', index=False)
    output_files[cat_type] = filename
    print(f"Saved {cat_type:12s}: {len(subset):6,} rows -> {filename}")

print("\n" + "="*50)
print("Files created for experiment:")
for cat_type, filename in output_files.items():
    print(f"  - {filename}")

# Download files if in Colab
if IN_COLAB:
    print("\nDownloading files...")
    for filename in output_files.values():
        files.download(filename)

## Additional Analysis: Error by Meaningful Subcategories

In [ ]:
# For meaningful categories, analyze top-level categories
df_meaningful = df_valid[df_valid['category_type'] == 'meaningful'].copy()

if len(df_meaningful) > 0:
    def get_top_category(cat):
        """Extract top-level category"""
        if pd.isna(cat):
            return 'Unknown'
        cat = str(cat)
        # Split by > or /
        if '>' in cat:
            return cat.split('>')[0].strip()
        elif '/' in cat:
            return cat.split('/')[0].strip()
        return cat.strip()

    df_meaningful['top_category'] = df_meaningful['category'].apply(get_top_category)

    # Top 15 categories by count
    top_cats = df_meaningful['top_category'].value_counts().head(15).index.tolist()

    # Calculate stats for top categories
    cat_stats = []
    for cat in top_cats:
        subset = df_meaningful[df_meaningful['top_category'] == cat]
        cat_stats.append({
            'Top Category': cat,
            'Count': len(subset),
            'Weight Median': subset[WEIGHT_ERROR].median(),
            'Volume Median': subset[VOLUME_ERROR].median(),
            'Combined Median': subset['combined_error'].median()
        })

    cat_stats_df = pd.DataFrame(cat_stats)
    print("\nTop 15 Categories (Meaningful) - Error Statistics:")
    print("=" * 80)
    print(cat_stats_df.round(4).to_string(index=False))
else:
    print("No meaningful categories with valid error data found.")

In [ ]:
# Visualize top categories
if len(df_meaningful) > 0 and len(top_cats) > 0:
    fig, ax = plt.subplots(figsize=(14, 8))

    x = np.arange(len(top_cats))
    width = 0.35

    weight_medians = [df_meaningful[df_meaningful['top_category']==cat][WEIGHT_ERROR].median() for cat in top_cats]
    volume_medians = [df_meaningful[df_meaningful['top_category']==cat][VOLUME_ERROR].median() for cat in top_cats]

    bars1 = ax.bar(x - width/2, weight_medians, width, label='Weight Error', color='#3498db', alpha=0.8)
    bars2 = ax.bar(x + width/2, volume_medians, width, label='Volume Error', color='#e74c3c', alpha=0.8)

    ax.set_ylabel('Median Error')
    ax.set_title('Median Error by Top Category (Meaningful Categories Only)', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(top_cats, rotation=45, ha='right')
    ax.legend()
    ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.savefig('top_category_error.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\nTop category chart saved as 'top_category_error.png'")
else:
    print("Skipping top category visualization - no data available.")

## Summary

### Key Findings:
1. **Category Distribution**: Meaningful categories have the majority of data
2. **Error Comparison**: Compare median errors across category types
3. **Separated Datasources**: Three files created for targeted experiments

### Output Files:
- `datasource_meaningful.tsv` - Items with valid product categories
- `datasource_malformed.tsv` - Items with invalid/navigation categories  
- `datasource_empty.tsv` - Items with no category
- `category_error_boxplot.png` - Box plot visualization
- `category_error_histogram.png` - Histogram and CDF
- `category_error_summary.png` - Min/Median/Max bar chart
- `top_category_error.png` - Top meaningful categories analysis